In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [17]:
n = 1000
frac = 270
temp_arr = np.random.rand(n)
fixed_arr = temp_arr[:n//2]
temp_indices = np.random.randint(0,n//2,frac)
X = fixed_arr[temp_indices]             #exisiting values

temp1 = np.random.randint(n//2,n,(n//2 - frac))
t1 = temp_arr[temp1]
X = np.append(X, t1)
X = X.astype("float32")

y = np.ones(frac)
y = np.append(y,np.zeros(n//2-frac))
y = y.astype("float32")


In [13]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = nn.Linear(1, n//2)
    self.fc1 = nn.Linear(n//2, 1)
  
  def forward(self, X):
    # print(X)
    Op_1 = F.relu(self.fc(X))
    Op_2 = self.fc1(Op_1)
    return F.relu(Op_2)
    # return F.log_softmax(self.fc(X),dim = 1)

  
net = Net()
net

Net(
  (fc): Linear(in_features=1, out_features=250, bias=True)
  (fc1): Linear(in_features=250, out_features=1, bias=True)
)

In [14]:
tau = 0.5

In [15]:
import torch.optim as optim
import tqdm as tq
optimizer = optim.Adam(net.parameters(),lr = 1e-2)
# search = torch.from_numpy(search)
# search = search.to("cuda")
train_input = torch.from_numpy(X)
train_ans = torch.from_numpy(y)
train_input = train_input.to('cuda')
train_ans = train_ans.to('cuda')

In [18]:
batch_size = 10
net.to('cuda')
epochs = 10
print("ok",n)
for e in tq.tqdm(range(epochs)):
  for b in range(int(n/batch_size)):
    
    x = train_input[b*batch_size:(b+1)*batch_size]
    y = train_ans[b*batch_size:(b+1)*batch_size]
    net.zero_grad()
    out = net(x.view(-1,1))
    # print(out.shape, y.view(-1,1).shape)
    loss= F.binary_cross_entropy(out,y.view(-1,1))
    loss.backward()
    optimizer.step()
  print(loss)

  # out = net(train_input.view(-1,n+1))
  # print(torch.mean(torch.square(out.argmax()-search).to(torch.float)))

ok 500


  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: ignored